In [7]:
# Installation of required libraries
!pip install librosa statsmodels pandas numpy

import librosa
import numpy as np
import pandas as pd
import statsmodels.api as sm
import glob
import os
import warnings
import time

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Configuration
FILE_EXTENSION = "*.flac"
LOAD_DURATION = 300   # Analyze the first 5 minutes (300s)
PITCH_DURATION = 60   # Limit pitch analysis to 60s for computational efficiency

def extract_golden_trio(file_path):
    """
    Extracts the 'Golden Trio' features:
    1. V_Pitch: Vibrato intensity (Standard Deviation of F0 via pYIN)
    2. V_Timbre: Tonal brightness (Spectral Centroid)
    3. V_Rhythm: Rhythmic flexibility (Standard Deviation of IBIs)
    """
    print(f"▶ Processing: {file_path}...")
    start_time = time.time()
    try:
        y, sr = librosa.load(file_path, sr=44100, duration=LOAD_DURATION)
    except Exception as e:
        print(f"!! Error loading {file_path}: {e}")
        return None

    # 1. V_Pitch (Vibrato/Intonation)
    f0, _, _ = librosa.pyin(y[:44100*PITCH_DURATION], fmin=130, fmax=1000)
    valid_f0 = f0[~np.isnan(f0)]
    if len(valid_f0) > 0:
        # Convert to cents relative to A440 and calculate STD
        v_pitch = np.std(1200 * np.log2(valid_f0 / 440.0))
    else:
        v_pitch = 0

    # 2. V_Timbre (Spectral Centroid)
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    v_timbre = np.mean(cent)

    # 3. V_Rhythm (Inter-Beat Interval deviation)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    if len(beat_times) > 1:
        v_rhythm = np.std(np.diff(beat_times))
    else:
        v_rhythm = 0

    elapsed = time.time() - start_time
    print(f"   -> Done ({elapsed:.1f}s). Pitch:{v_pitch:.2f}, Timbre:{v_timbre:.0f}, Rhythm:{v_rhythm:.4f}")
    return [v_pitch, v_timbre, v_rhythm]

# Main Execution Pipeline
files = sorted(glob.glob(FILE_EXTENSION))
print(f"Found {len(files)} files: {files}")

if len(files) < 4:
    print("!! Error: Insufficient sample size. Please upload at least 4 .flac files.")
else:
    # Feature Extraction Loop
    data = []
    indices = []
    print(f"\n[Analyzing {LOAD_DURATION} seconds per file...]")
    for f in files:
        feats = extract_golden_trio(f)
        if feats:
            data.append(feats)
            year = ''.join(filter(str.isdigit, f))
            indices.append(year)

    # DataFrame Construction & Standardization
    df = pd.DataFrame(data, columns=['V_Pitch', 'V_Timbre', 'V_Rhythm'], index=indices)
    df_std = (df - df.mean()) / df.std()
    df_std = df_std.fillna(0)

    print("\n" + "="*40)
    print(" [Extracted Data (Standardized)] ")
    print("="*40)
    print(df_std)

    # ---------------------------------------------------------
    # PART 1: Model Competition (The "Gauntlet")
    # ---------------------------------------------------------
    print("\n" + "="*40)
    print(" [Step 1: Model Selection via AIC] ")
    print("="*40)

    def run_ssm(data, k):
        try:
            # Diagonal error covariance for stability with small N
            model = sm.tsa.DynamicFactor(data, k_factors=k, factor_order=1, error_var='diagonal')
            res = model.fit(disp=False, maxiter=3000)
            return res, res.aic
        except:
            return None, np.inf

    # Run both models blindly
    res_k1, aic1 = run_ssm(df_std, k=1)
    res_k2, aic2 = run_ssm(df_std, k=2)

    print(f"▶ Model A (k=1, Single Factor)      | AIC: {aic1:.4f}")
    print(f"▶ Model B (k=2, Thesis-Antithesis)  | AIC: {aic2:.4f}")

    # ---------------------------------------------------------
    # PART 2: Algorithmic Decision & Semantic Validation
    # ---------------------------------------------------------
    print("\n" + "="*40)
    print(" [Step 2: Automated Analysis of the Winner] ")
    print("="*40)

    # Let the DATA decide the winner, not me (researcher)
    if aic2 < aic1:
        best_k = 2
        best_model_res = res_k2
        print(f">> Decision: The Algorithm selected Model B (k=2) as the optimal structure.")
    else:
        best_k = 1
        best_model_res = res_k1
        print(f">> Decision: The Algorithm selected Model A (k=1).")

    print(f">> Proceeding with Semantic Validation for k={best_k}...")

    # Validate Factor Loadings for the WINNER only
    try:
        print("\n>>> Optimal Model Summary (Inspect 'loadings'):")
        print(best_model_res.summary())

        if best_k == 2:
            print("\n" + "-"*40)
            print(" [Interpretation Guide for k=2] ")
            print("-"*40)
            print("1. Factor 1 Loadings: Correlations with V_Pitch/V_Timbre (Romanticism)")
            print("2. Factor 2 Loadings: Correlations with V_Rhythm (Objectivism)")
            print("Note: Strong loadings confirm the 'Thesis-Antithesis' narrative.")
    except Exception as e:
        print(f"Error in Analysis: {e}")


Found 4 files: ['heifetz_1952.flac', 'perlman_1998.flac', 'shaham_2015.flac', 'szeryng_1967.flac']

[Analyzing 300 seconds per file...]
▶ Processing: heifetz_1952.flac...
   -> Done (18.2s). Pitch:481.08, Timbre:2470, Rhythm:0.0264
▶ Processing: perlman_1998.flac...
   -> Done (18.5s). Pitch:497.14, Timbre:2139, Rhythm:0.0281
▶ Processing: shaham_2015.flac...
   -> Done (27.3s). Pitch:607.02, Timbre:2393, Rhythm:0.0551
▶ Processing: szeryng_1967.flac...
   -> Done (19.9s). Pitch:437.35, Timbre:2219, Rhythm:0.0309

 [Extracted Data (Standardized)] 
       V_Pitch  V_Timbre  V_Rhythm
1952 -0.340480  1.080756 -0.646623
1998 -0.117976 -1.087275 -0.525285
2015  1.405039  0.573249  1.485751
1967 -0.946583 -0.566731 -0.313843

 [Step 1: Model Selection via AIC] 
▶ Model A (k=1, Single Factor)      | AIC: 36.5147
▶ Model B (k=2, Thesis-Antithesis)  | AIC: 28.2994

 [Step 2: Automated Analysis of the Winner] 
>> Decision: The Algorithm selected Model B (k=2) as the optimal structure.
>> Proceed